In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_csv('merged_A61.csv')

In [ ]:
df.head()

,patent_id,patent_abstract,patent_title,claim_text,cpc_subclass,generated_abstract
0,11213381,A prosthetic capsular device configured to be ...,"Prosthetic capsular devices, systems, and methods",A prosthetic capsular system for insertion int...,A61F,The present invention relates to a prostheti...
1,11246717,A method and system for performing bone fusion...,Expandable intervertebral cage,An intervertebral cage comprising: a first sup...,A61F,"An intervertebral cage is described, compris..."
2,11234638,An oral appliance for the treatment of sleep d...,Intra-oral electroencephalography device and m...,An oral appliance for treatment of sleep apnea...,A61N,The present invention relates to an oral app...
3,11304855,Provided are buffered adhesive compositions co...,Buffered adhesive compositions for skin-adheri...,An ostomy skin barrier comprising: a first pre...,A61L,The present invention relates to an ostomy s...
4,11369552,An aqueous composition for removing nail polis...,Aqueous nail polish remover,An aqueous composition for nail polish removal...,A61Q,The present invention relates to an aqueous ...


In [ ]:
import os
from openai import OpenAI

In [ ]:
import pandas as pd
def generate_key(abstract):
    client = OpenAI(
        api_key="######" #API key from OpenAI
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "assistant",
                "content": f"Given the patent abstract write some keyphrases separated by comma. Don't inclue any other info. Abstract: {abstract}",
            }
        ],
        model="gpt-3.5-turbo",
    )
    abstract = chat_completion.choices[0].message.content
    return abstract

In [ ]:
df1 = df.iloc[0:1000].copy()

df1['keyphrase'] = pd.NA
df1['gen_keyphrase'] = pd.NA
df1.iloc[0:0].to_csv('Generated_keyphrase_A61.csv', index=False)

for index, row in df1.iterrows():
    print(index)
    key = generate_key(row['patent_abstract'])
    gen_key = generate_key(row['generated_abstract'])
    print(key)
    print(gen_key)
    df1.at[index, 'keyphrase'] = key if isinstance(key, str) else 'Error or empty'
    df1.at[index, 'gen_keyphrase'] = gen_key if isinstance(gen_key, str) else 'Error or empty'

    # Append the row without the header
    df1.iloc[[df1.index.get_loc(index)]].to_csv('Generated_keyphrase_A61.csv', mode='a', header=False, index=False)

0
prosthetic capsular device, eye, housing structure, ring structure, intraocular device, IOL, refractive surface, longitudinal axis, cavity, opening, ring structure portion
prosthetic capsular system, natural capsular bag, intraocular lenses, housing structure, anterior arcuate opening, textured surface, exterior sidewall, secure and stable fit, interior grooves, easy insertion, removal, replacement
1
method, system, bone fusion, bone securing, vertebra
intervertebral cage, support member, spacer, drive mechanism, pivotable engagement, lateral movement, contact surface
2
oral appliance, sleep disorders, obstructive sleep apnea, mouthpiece, dentition, oxygen sensor, pressure sensor, airflow sensor, actigraphy sensor, noise detector, stimulator, tongue stimulation, decreased oxygen saturation levels, increased pressure, decreased airflow, increased noise levels, EEG sensor, sleep state, pharmaceutical compound reservoirs, oral mucosa, microprocessor, data activation
oral appliance, slee

## Evaluation

In [ ]:
import pandas as pd

def calculate_metrics(generated, original):
    """
    Calculate precision, recall, and F1 score between generated and original keyphrases.

    Args:
    generated (list of str): The  keyphrases from generated abstracts.
    original (list of str):  The  keyphrases from original abstracts.

    Returns:
    tuple: A tuple containing precision, recall, and F1 score.
    """
    generated_set = set(generated)
    original_set = set(original)

    correct = generated_set.intersection(original_set)

    recall = len(correct) / len(original_set) if original_set else 0
    precision = len(correct) / len(generated_set) if generated_set else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    return precision, recall, f1_score


df = pd.read_csv('/content/Generated_keyphrase_A61.csv')

total_precision, total_recall, total_f1_score = 0, 0, 0


for _, row in df.iterrows():
        p, r, f1 = calculate_metrics(row['gen_keyphrase'], row['keyphrase'])
        total_precision += p
        total_recall += r
        total_f1_score += f1

avg_precision = total_precision / len(df)
avg_recall = total_recall / len(df)
avg_f1_score = total_f1_score / len(df)

print(f"Average Precision: {avg_precision:.2f}")
print(f"Average Recall: {avg_recall:.2f}")
print(f"Average F1 Score: {avg_f1_score:.2f}")
